### Imports

In [1]:
import pandas as pd
import numpy as np
from tqdm import tqdm
tqdm.pandas(desc="progress-bar")
from gensim.models import Doc2Vec
from sklearn import utils
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import roc_auc_score
from sklearn.metrics import accuracy_score
from sklearn.metrics import classification_report
from sklearn.model_selection import train_test_split
import gensim
import gensim.models.doc2vec as doc2vec
from gensim.models.doc2vec import TaggedDocument
import re

### Load data & clean it up

In [206]:
df = pd.read_csv('/Users/ksmith/Documents/Code/DS1/Unit4_Project/build-post-here-DS/plz_work.csv')
df = df.fillna('')
df['all_text'] = df['title'] + ' ' + df['text'] # Combining text & titles to one field

df.head(10)

,Unnamed: 0,submission_id,title,text,subreddit_id,name,all_text
0,0,bcy7ky,15 minutes wait to play more?,"So I dodged a game, all my mains banned or picked. I could have played a champ I don't know well, and cost my team a loss anyway. This is pretty rare for all my main champs to be picked/banned, so much so that I havn't dodged a game in months. More than I even remember, honestly it could be a year or more. \r\r\n\r\r\nSo here I am supposed to wait 15 mins for an online game. Is this ok? I'm trying to figure out what the logic behind this kind of punishment is.",2rfxx,leagueoflegends,"15 minutes wait to play more? So I dodged a game, all my mains banned or picked. I could have played a champ I don't know well, and cost my team a loss anyway. This is pretty rare for all my main champs to be picked/banned, so much so that I havn't dodged a game in months. More than I even remember, honestly it could be a year or more. \r\r\n\r\r\nSo here I am supposed to wait 15 mins for an online game. Is this ok? I'm trying to figure out what the logic behind this kind of punishment is."
1,1,bcy6mf,League has to investigate possible fake game by Zen.,Obvious inting in game 5 right before TSM win the game. Intentionally de-morale the whole team and let TL come back.\r\r\n\r\r\nNo one is saying he got paid by gambling company or enemy team. But some investigation has to be done. This is just too strange to believe.,2rfxx,leagueoflegends,League has to investigate possible fake game by Zen. Obvious inting in game 5 right before TSM win the game. Intentionally de-morale the whole team and let TL come back.\r\r\n\r\r\nNo one is saying he got paid by gambling company or enemy team. But some investigation has to be done. This is just too strange to believe.
2,2,bcz8v5,"Xmithie...the Goat, I told you so","Here's what I said about Xmithie years ago...\r\r\n\r\r\nIf I'm not mistaken, he's taken more different teams to worlds than anyone in LoL history..\r\r\n\r\r\nGrats to the GOAT... the best jungled ever to play in NA\r\r\n\r\r\nhttps://www.reddit.com/r/leagueoflegends/comments/3rnrgu/xmithieunderrated/",2rfxx,leagueoflegends,"Xmithie...the Goat, I told you so Here's what I said about Xmithie years ago...\r\r\n\r\r\nIf I'm not mistaken, he's taken more different teams to worlds than anyone in LoL history..\r\r\n\r\r\nGrats to the GOAT... the best jungled ever to play in NA\r\r\n\r\r\nhttps://www.reddit.com/r/leagueoflegends/comments/3rnrgu/xmithieunderrated/"
3,3,bcz9r2,Whatever happened to that Teacher in Korea who would make lol related posts about his students,Anyone remember this ? It was way back like 2014-15 I believe .,2rfxx,leagueoflegends,Whatever happened to that Teacher in Korea who would make lol related posts about his students Anyone remember this ? It was way back like 2014-15 I believe .
4,4,bcz9wj,Just got to diamond playing annie only :),"Just wanted to share my excitement with you guys. I only played annie (except for 2 games) and reached diamond.\r\r\n\r\r\nIf you guys have any questions feel free to ask me, I am down to share some tips if you guys want.\r\r\n\r\r\nhttps://i.redd.it/gq9wku6vr5s21.png",2rfxx,leagueoflegends,"Just got to diamond playing annie only :) Just wanted to share my excitement with you guys. I only played annie (except for 2 games) and reached diamond.\r\r\n\r\r\nIf you guys have any questions feel free to ask me, I am down to share some tips if you guys want.\r\r\n\r\r\nhttps://i.redd.it/gq9wku6vr5s21.png"
5,5,bcza0d,Hot Take: TSM vs TL is the Best Finals in LCS History,"As a TSM fan it sucks to lose, but those games were so fuckin hype and crazy. Both teams played out of their fuckin minds. The only other ones that seemed to stand out to me are TSM vs C9 2016/17 and TSM vs CLG 2015/15. The crowd was also really good St. Louis did not disappoint. They seemed to be back and forth with chants non-stop all series. what do you people think? Was this the best LCS finals?",2rfxx,leagueoflegends,"Hot Take: TSM vs

In [227]:
top_20 = ['AskReddit', 'dankmemes', 'memes', 'teenagers', 'aww', 'RocketLeagueExchange', 'Showerthoughts',
          'funny', 'me_irl', 'freefolk', 'gameofthrones', 'pics', 'NoStupidQuestions', 'AskOuija',
          'unpopularopinion', 'gaming', 'videos', 'politics', 'AmItheAsshole', 'Jokes']

data = df[df['name'].isin(top_20)]
data = data.drop(data.columns[0], axis=1) # Drop old row index
data.head()

,submission_id,title,text,subreddit_id,name,all_text
551,bcy7k8,"[Spoilers] What is that one conspiracy theory you have that noone agrees with, but you still think is going to happen... ?",Arya is already dead and is actually Jaqen H'ghar...,2rjz2,gameofthrones,"[Spoilers] What is that one conspiracy theory you have that noone agrees with, but you still think is going to happen... ? Arya is already dead and is actually Jaqen H'ghar..."
552,bcz9cw,[SPOILERS] The only reunion that I want to see tonight.,,2rjz2,gameofthrones,[SPOILERS] The only reunion that I want to see tonight.
553,bczaow,[No Spoilers] How does rising the dead work?,Some stuff I wonder about.\r\r\n\r\r\nHow fresh do the dead need to be? Can the NK rise people ho died weeks/months/years ago?\r\r\n\r\r\nAnd how close does he have to be? Could he rise people in Dorne while he sits somewhere in the Riverlands?\r\r\n\r\r\nWhere those rules ever established?,2rjz2,gameofthrones,[No Spoilers] How does rising the dead work? Some stuff I wonder about.\r\r\n\r\r\nHow fresh do the dead need to be? Can the NK rise people ho died weeks/months/years ago?\r\r\n\r\r\nAnd how close does he have to be? Could he rise people in Dorne while he sits somewhere in the Riverlands?\r\r\n\r\r\nWhere those rules ever established?
554,bczgts,[No Spoilers] House Targaryen sigil cross stitch,,2rjz2,gameofthrones,[No Spoilers] House Targaryen sigil cross stitch
555,bczhln,[No Spoilers] Need help!!,"My exams(very important) are going on and will end on 27th May, which is way past the final episode release date. Which is a problem because now I can't decide whether to watch one episode per week of the new season or wait till after the exams and then watch all 8 seasons together. I really want to watch all the seasons again so I can do justice to the series but I don't know if I will be safe from spoilers for so long. I need help deciding!!",2rjz2,gameofthrones,"[No Spoilers] Need help!! My exams(very important) are going on and will end on 27th May, which is way past the final episode release date. Which is a problem because now I can't decide whether to watch one episode per week of the new season or wait till after the exams and then watch all 8 seasons together. I really want to watch all the seasons again so I can do justice to the series but I don't know if I will be safe from spoilers for so long. I need help deciding!!"


In [208]:
from nltk.corpus import stopwords, wordnet
import string

stop_words = set(stopwords.words('english'))
table = str.maketrans('', '', string.punctuation)

data['all_text'] = data['all_text'].str.replace('http\S+|www.\S+', '', case=False)
data['all_text'] = data['all_text'].str.lower()  #Text is lowercase
data['all_text'] = data['all_text'].str.translate(table) #Remove punctuation
# data['all_text'] = data['all_text'].str.split() #Tokenize words
# data['all_text'] = data['all_text'].apply(lambda x: [item for item in x if item not in stop_words]) 
# #Remove stop words

In [204]:
data['all_text'].str.findall('.*\://(?:www.)?([^\/]+)')

551                                        []
552                                        []
553                                        []
554                                        []
555                                        []
556                                        []
557                                        []
558                                        []
559                                        []
560                                        []
561                                        []
562                                        []
563      [github.com, youtube.com, i.redd.it]
564                                        []
565                                        []
566                             [youtube.com]
567                                        []
568                                        []
569                                        []
570                                        []
571                                        []
572                               

In [231]:
pd.options.display.max_colwidth = 500

# Using regex groups
pat = r'http\S+|www.\S+'

def repl(m):
    return r'.*\://(?:www.)?([^\/]+)'
 

data['all_text'].str.replace(pat, repl)

551                                                                                                                                                                                                                                                                                                                                           [Spoilers] What is that one conspiracy theory you have that noone agrees with, but you still think is going to happen... ? Arya is already dead and is actually Jaqen H'ghar...
552                                                                                                                                                                                                                                                                                                                                                                                                                                                                  [SPOILERS] The only reunion that I wa

### Doc2Vec & Logistic Regression

In [209]:
def label_sentences(corpus, label_type):
    """
    Gensim's Doc2Vec implementation requires each document/paragraph to have a label associated with it.
    We do this by using the TaggedDocument method. The format will be "TRAIN_i" or "TEST_i" where "i" is
    a dummy index of the post.
    """
    labeled = []
    for i, v in enumerate(corpus):
        label = label_type + '_' + str(i)
        labeled.append(doc2vec.TaggedDocument(v.split(), [label]))
    return labeled

In [233]:
# y_encode = {'AskReddit':1, 'dankmemes':2, 'memes':3, 'teenagers':4, 'aww':5, 'RocketLeagueExchange':6, 
#             'Showerthoughts':7,'funny':8, 'me_irl':9, 'freefolk':10, 'gameofthrones':11, 'pics':12, 
#             'NoStupidQuestions':13, 'AskOuija':14, 'unpopularopinion':15, 'gaming':16, 'videos':17, 
#             'politics':18, 'AmItheAsshole':19, 'Jokes':20}

X = data['all_text']
y = data['name']

AskReddit               18286
NoStupidQuestions        5888
politics                 5648
Jokes                    4865
funny                    4745
pics                     4418
videos                   4382
aww                      4333
gaming                   4257
teenagers                3442
RocketLeagueExchange     3380
me_irl                   3279
freefolk                 3276
gameofthrones            3106
Showerthoughts           3070
AmItheAsshole            2480
AskOuija                 2475
dankmemes                2446
memes                    2217
unpopularopinion         1997
Name: name, dtype: int64

In [211]:
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=0, test_size=0.3)
X_train = label_sentences(X_train, 'Train')
X_test = label_sentences(X_test, 'Test')
all_data = X_train + X_test

In [212]:
all_data[:2]

[TaggedDocument(words=['whats', 'the', 'most', 'embarrassing', 'thing', 'youve', 'done', 'in', 'front', 'of', 'a', 'crush'], tags=['Train_0']),
 TaggedDocument(words=['no', 'spoilers', 'what', 'will', 'you', 'do', 'for', 'the', 'throne', 'you', 'can', 'see', 'my', 'illustration', 'of', 'daenerys', 'there', 'for', 'a', 'split', 'second', '030', 'and', 'other', 'amazing', 'creations'], tags=['Train_1'])]

In [213]:
model_dbow = Doc2Vec(dm=0, vector_size=300, negative=5, min_count=1, alpha=0.065, min_alpha=0.065)
model_dbow.build_vocab([x for x in tqdm(all_data)])

for epoch in range(30):
    model_dbow.train(utils.shuffle([x for x in tqdm(all_data)]), total_examples=len(all_data), epochs=1)
    model_dbow.alpha -= 0.002
    model_dbow.min_alpha = model_dbow.alpha

100%|██████████| 87990/87990 [00:00<00:00, 3340817.87it/s]


In [214]:
def get_vectors(model, corpus_size, vectors_size, vectors_type):
    """
    Get vectors from trained doc2vec model
    :param doc2vec_model: Trained Doc2Vec model
    :param corpus_size: Size of the data
    :param vectors_size: Size of the embedding vectors
    :param vectors_type: Training or Testing vectors
    :return: list of vectors
    """
    vectors = np.zeros((corpus_size, vectors_size))
    for i in range(0, corpus_size):
        prefix = vectors_type + '_' + str(i)
        vectors[i] = model.docvecs[prefix]
    return vectors
    
train_vectors_dbow = get_vectors(model_dbow, len(X_train), 300, 'Train')
test_vectors_dbow = get_vectors(model_dbow, len(X_test), 300, 'Test')

In [215]:
subreddits = data.name.unique()

In [216]:
%%time

logreg = LogisticRegression(n_jobs=1, C=1e5)
logreg = logreg.fit(train_vectors_dbow, y_train)

CPU times: user 1min 41s, sys: 257 ms, total: 1min 41s
Wall time: 1min 42s


In [217]:
y_pred = logreg.predict(test_vectors_dbow)
print('accuracy %s' % accuracy_score(y_pred, y_test))
print(classification_report(y_test, y_pred,target_names=subreddits))

accuracy 0.6274955487365989
                      precision    recall  f1-score   support

       gameofthrones       0.88      0.90      0.89       733
           teenagers       0.34      0.12      0.18       755
           AskReddit       0.75      0.94      0.84      5579
                pics       0.54      0.50      0.52      1494
           dankmemes       0.58      0.59      0.58      1679
RocketLeagueExchange       0.96      0.96      0.96       999
                 aww       0.48      0.37      0.42       889
              me_irl       0.52      0.64      0.58      1214
               funny       0.29      0.05      0.09       771
      Showerthoughts       0.53      0.48      0.50       958
               memes       0.26      0.33      0.29      1448
    unpopularopinion       0.76      0.77      0.77       910
       AmItheAsshole       0.49      0.52      0.51      1253
              videos       0.75      0.94      0.83      1031
            freefolk       0.20      0.01

### User input test prediction

In [218]:
input_title = ["Python tutorial"]
input_text = ["I'm looking for a good python tutorial for free online. Any suggestions?"]
all_input = input_title + input_text
user_input = model_dbow.infer_vector(all_input, steps=20)
user_input = user_input.reshape(1, -1)
prediction = logreg.predict(user_input)
print(prediction)

['me_irl']


In [134]:
# ivd = {v: k for k, v in y_encode.items()}
# [ivd[x] for x in prediction]

In [219]:
output_df = pd.DataFrame(logreg.predict_proba(user_input), columns=logreg.classes_).T
top_5 = output_df.nlargest(5, [0])
top_5.reset_index().values

array([['me_irl', 0.14852466047962495],
       ['funny', 0.13061337494682707],
       ['dankmemes', 0.12219000507000331],
       ['memes', 0.12099509447774118],
       ['aww', 0.11893742723244544]], dtype=object)

In [220]:
output = pd.DataFrame(logreg.predict_proba(user_input), columns=logreg.classes_).T.nlargest(5, [0]).reset_index().values
print(output)

[['me_irl' 0.14852466047962495]
 ['funny' 0.13061337494682707]
 ['dankmemes' 0.12219000507000331]
 ['memes' 0.12099509447774118]
 ['aww' 0.11893742723244544]]
